In [1]:
import cv2
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.preprocessing import LabelEncoder

In [2]:
#load the models
linearRegression_Age = tf.keras.models.load_model('model_checkpoint1.h5')
binaryClass_Identity = tf.keras.models.load_model('model_checkpoint2.h5')
multiClass_Expression = tf.keras.models.load_model('model_checkpoint3.h5')

In [3]:
def preprocess(frame):
    resized_frame = cv2.resize(frame, (32, 32))
    flattened_frame = np.reshape(resized_frame, (1, -1))
    return flattened_frame/255.

In [4]:
cv2.namedWindow("preview") #creates a window named "preview" that will be used to display the video feed.
vc = cv2.VideoCapture(0) #creates a VideoCapture object, which is used to capture video from a camera. 
#The argument 0 specifies the camera index, meaning that the script will use the default camera on the computer.

if vc.isOpened(): #checks if the camera is successfully opened
    rval, frame = vc.read()
else:
    rval = False

while rval:
    frame_processed = preprocess(frame)
    
    prediction = binaryClass_Identity.predict(frame_processed)
    label_encoder = LabelEncoder()
    label_encoder.classes_ = np.array(['Hamza Ahmad', 'Unknown'])
    label_index = int(np.squeeze(prediction).round())
    decoded_prediction = label_encoder.classes_[label_index]
    
    # Apply threshold to classify as Hamza Ahmad or Unknown
    threshold = 0.5
    identity = "Hamza Ahmad" if prediction <= threshold else "Unknown"
    
    age = linearRegression_Age.predict(frame_processed)
    expr = multiClass_Expression.predict(frame_processed)
    
    expr_label = np.argmax(expr)
    if expr_label == 0:
        expr_text = "Angry"
    elif expr_label == 1:
        expr_text = "Not Smiling"
    elif expr_label == 2:
        expr_text = "Smiling"
    else:
        expr_text = "Unknown"
    
    # Display the age predictions alongside the video feed
    cv2.putText(frame, f"Identity: {identity}", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2, cv2.LINE_AA)
    cv2.putText(frame, f"Age: {age}", (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2, cv2.LINE_AA)
    cv2.putText(frame, f"Expression: {expr_text}", (50, 150), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2, cv2.LINE_AA)
    
    
    
    #This loop captures frames of video from the camera and displays them in the "preview" window.
    cv2.imshow("preview", frame) #displays the frame in the "preview" window
    rval, frame = vc.read() #reads a frame from the camera, and the resulting image is stored in the frame variable
    
    key = cv2.waitKey(20) #waits for a key to be pressed for 20 milliseconds
    if key == 27: # exit on ESC
        break

vc.release() #releases the camera
cv2.destroyWindow("preview") #destroys the "preview" window.

1/1 [==============================] - 0s 39ms/step


1/1 [==============================] - 0s 39ms/step
